In [2]:
from particletracking import dataframes
import trackpy as tp
from tqdm import tqdm
import numpy as np
import filehandling

In [3]:
flat_files = filehandling.get_directory_filenames("/media/NAS/share/James/HighSpeed/Flat/*")
dimpled_files = filehandling.get_directory_filenames("/media/NAS/share/James/HighSpeed/DimpledNew/*")

In [4]:
flat_files = [d for d in flat_files if d.endswith("0.hdf5")]
dimpled_files = [d for d in dimpled_files if d.endswith("0.hdf5")]

In [5]:
dimpled_files

['/media/NAS/share/James/HighSpeed/DimpledNew/520_frames_1_1000.hdf5',
 '/media/NAS/share/James/HighSpeed/DimpledNew/540_frames_1_1000.hdf5',
 '/media/NAS/share/James/HighSpeed/DimpledNew/560_frames_1_1000.hdf5',
 '/media/NAS/share/James/HighSpeed/DimpledNew/580_frames_1_1000.hdf5',
 '/media/NAS/share/James/HighSpeed/DimpledNew/600_frames_1_1000.hdf5',
 '/media/NAS/share/James/HighSpeed/DimpledNew/620_frames_1_1000.hdf5',
 '/media/NAS/share/James/HighSpeed/DimpledNew/640_frames_1_1000.hdf5',
 '/media/NAS/share/James/HighSpeed/DimpledNew/660_frames_1_1000.hdf5',
 '/media/NAS/share/James/HighSpeed/DimpledNew/680_frames_1_1000.hdf5',
 '/media/NAS/share/James/HighSpeed/DimpledNew/700_frames_1_1000.hdf5']

In [13]:
import os

In [14]:
flat_duties = [int(os.path.split(d)[1][:3]) for d in flat_files]
dimpled_duties = [int(os.path.split(d)[1][:3]) for d in dimpled_files]

In [15]:
flat_datas = [dataframes.DataStore(d) for d in flat_files]
dimpled_datas = [dataframes.DataStore(d) for d in dimpled_files]

In [18]:
flat_datas[0].df.frame.max()

998

In [19]:
def get_mean_velocity(data, fps):
    displacements = []
    N = data.df.frame.max()
    for f in tqdm(range(1, N)):
        d = tp.relate_frames(data.df, f, f-1)
        d.dr = np.abs(d.dr)
        mean_disp = d.dr.mean()
        displacements.append(mean_disp)
        
    return np.nanmean(displacements) / fps, np.nanstd(displacements) / fps

In [20]:
flat_results = [get_mean_velocity(d, 500) for d in flat_datas]
dimpled_results = [get_mean_velocity(d, 500) for d in dimpled_datas]

100%|██████████| 997/997 [00:11<00:00, 85.41it/s] 


In [21]:
dimpled_velocities, dimpled_errors = list(zip(*dimpled_results))
flat_velocities, flat_errors = list(zip(*flat_results))


In [22]:
m = 3.87e-5 # kg

In [25]:
import matplotlib.pyplot as plt

In [26]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [27]:
dimpled_ke = 0.5 * m * np.array(dimpled_velocities)**2
flat_ke = 0.5 * m * np.array(flat_velocities)**2
dimple_ke_err = np.array(dimpled_errors)/np.array(dimpled_velocities) * dimpled_ke
flat_ke_err = np.array(flat_errors)/np.array(flat_velocities) * flat_ke

In [28]:
plt.errorbar(flat_duties, flat_ke, flat_ke_err, fmt='x', label='flat')
plt.errorbar(dimpled_duties, dimpled_ke, dimple_ke_err, fmt='x', label='dimpled')
plt.xlabel('Duty Cycle')
plt.ylabel('Total Kinetic Energy (J) ')
plt.legend()

In [49]:
mpp = 4e3/dimpled_datas[0].df.r.mean()
mpp

79.4899186646237

In [174]:
def get_mean_displacements(data, fps, mps):
    msd = tp.motion.imsd(data.df, mps, fps)
    print(msd.head())
    

In [178]:
tp.plot_traj(dimpled_datas[0].df)

<AxesSubplot:xlabel='x [px]', ylabel='y [px]'>

In [180]:

tp.plot_traj(flat_datas[0].df)

<AxesSubplot:xlabel='x [px]', ylabel='y [px]'>

In [182]:
dimpled_duties

[520, 540, 560, 580, 600, 620, 640, 660, 680, 700]

In [31]:
d560 = dimpled_datas[3].df

In [32]:
mean_dr = [tp.relate_frames(d560, 0, f).dr.mean()/f for f in range(1, 998)]

In [33]:
plt.plot(np.arange(1, 998), mean_dr)

In [45]:
dr_all = []
dr_errs = []
for f in np.arange(5, 150, 5):
    print(f)
    s = 0
    e = s + f
    dr = np.array([])
    while e < 998:
        dr = np.append(dr, tp.relate_frames(d560, s, e).dr/(e-s))
        s += f
        e += f
    plt.hist(dr, bins=100)

5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145


In [44]:
plt.errorbar(np.arange(5, 150, 5), dr_all, dr_errs)

<ErrorbarContainer object of 3 artists>

In [47]:
tp.relate_frames(d560, 1, 50)

,x,y,x_b,y_b,dx,dy,dr,direction
particle,,,,,,,,
0,621.5,371.5,738.5,281.5,117.0,-90.0,147.610977,-0.655696
1,1126.5,38.5,1170.5,21.5,44.0,-17.0,47.169907,-0.368696
2,416.5,78.5,428.5,69.5,12.0,-9.0,15.000000,-0.643501
3,307.5,310.5,256.5,393.5,-51.0,83.0,97.416634,2.121779
4,947.5,747.5,1006.5,767.5,59.0,20.0,62.297672,0.326827
5,445.5,952.5,361.5,917.5,-84.0,-35.0,91.000000,-2.746802
6,526.5,713.5,504.5,732.5,-22.0,19.0,29.068884,2.429235
7,1008.5,384.5,1112.5,420.5,104.0,36.0,110.054535,0.333244
8,1123.5,233.5,1101.5,142.5,-22.0,-91.0,93.621582,-1.808003


In [54]:
for data, duty in zip(dimpled_datas, dimpled_duties):
    tp.motion.emsd(data.df, mpp, 500).plot(label=duty)
plt.legend()